In [1]:
# imports
import sys, os

import random
import time

from argparse import ArgumentParser
import pickle

import numpy as np
import torch as th
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch import Tensor

import gym
import minerl

#from data_loader import DataLoader
from lib.tree_util import tree_map

if th.cuda.is_available():
    DEVICE = th.device('cuda')
    print("USING CUDA")
else:
    DEVICE = th.device('cpu')
    print("USING CPU")

USING CPU


In [2]:
def load_model_parameters(path_to_model_file):
    agent_parameters = pickle.load(open(path_to_model_file, "rb"))
    policy_kwargs = agent_parameters["model"]["args"]["net"]["args"]
    pi_head_kwargs = agent_parameters["model"]["args"]["pi_head_opts"]
    pi_head_kwargs["temperature"] = float(pi_head_kwargs["temperature"])

    return policy_kwargs, pi_head_kwargs

In [3]:
from agent import PI_HEAD_KWARGS, MineRLAgent

agent_policy_kwargs, agent_pi_head_kwargs = load_model_parameters("2x.model")

env = gym.make("MineRLBasaltFindCave-v0")
agent = MineRLAgent(env, device=DEVICE, policy_kwargs=agent_policy_kwargs, pi_head_kwargs=agent_pi_head_kwargs)
env.close()
policy = agent.policy
trainable_parameters = policy.parameters()

print(policy.net.hidsize)
print(agent_policy_kwargs)


#frames = agent._video_obs_to_agent(frames)
#print(agent.predict_actions(np.repeat(frames,1,axis=0)))
#print(agent.hidden_state[0])

from agent import PI_HEAD_KWARGS, MineRLAgent



2048
{'active_reward_monitors': {'craft_stats': {'args': {'collapse_var': True, 'items': ['planks', 'stick', 'crafting_table', 'wooden_pickaxe', 'stone_pickaxe', 'furnace', 'iron_ingot', 'iron_pickaxe', 'diamond_pickaxe', 'torch']}, 'weight': 0}, 'mine_stats': {'args': {'collapse_var': True, 'items': ['log', 'coal_ore', 'stone', 'iron_ore', 'diamond_ore', 'obsidian']}, 'weight': 0}, 'order_invariant_curriculum': {'args': {'curriculum': {'coal': [5, 0.4], 'cobblestone': [11, 0.09090909090909091], 'crafting_table': [1, 1], 'diamond': [10000, 2.6666666666666665], 'diamond_pickaxe': [10000, 8], 'furnace': [1, 1], 'iron_ingot': [3, 1.3333333333333333], 'iron_ore': [3, 1.3333333333333333], 'iron_pickaxe': [1, 4], 'log': [8, 0.125], 'obsidian': [10000, 16], 'planks': [20, 0.05], 'stick': [16, 0.0625], 'stone_pickaxe': [1, 1], 'torch': [16, 0.125], 'wooden_pickaxe': [1, 1]}}, 'weight': 1}, 'pickup_stats': {'args': {'collapse_var': True, 'items': ['log', 'coal', 'cobblestone', 'iron_ore', 'diam

In [4]:
frameA = th.normal(0,1,[1,1,128,128,3])
frameB = th.normal(0,1,[1,1,128,128,3])
tokenA = th.full([1,1],2, dtype=th.long)
tokenB = th.full([1,1],4127, dtype=th.long)

# get output for input P(A)
frames = {'img':frameA,
            'ms': th.tensor([0,0])}

words = {'token_ids':tokenA,
            'ms': th.tensor([0,0])}
pd_action, vpred_action, pd_word, VPT_state_out = agent.policy.get_output_for_observations(ob_frames=frames, ob_words=words)
print(pd_action)



# get output for input P(A)->state->P(B)
frames = {'img':frameB,
            'ms': th.tensor([0,0])}

words = {'token_ids':tokenB,
            'ms': th.tensor([0,0])}
pd_action, vpred_action, pd_word, VPT_state_out = agent.policy.get_output_for_observations(ob_frames=frames, ob_words=words, VPT_state_in=VPT_state_out)
print(pd_action)









# get output for P(A,B)
frames = {'img':th.cat([frameA,frameB], axis=1),
            'ms': th.tensor([0,0])}

words = {'token_ids':th.cat([tokenA,tokenB], axis=1),
            'ms': th.tensor([0,0])}
pd_action, vpred_action, pd_word, VPT_state_out = agent.policy.get_output_for_observations(ob_frames=frames, ob_words=words)
print(pd_action)



words, frmes=1 1


/home/idmi/miniconda3/envs/minerl/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


VPT1 out shape torch.Size([1, 1, 2048])
xattn1 out shape torch.Size([1, 1, 1024])
LM out shape torch.Size([1, 1, 1024])
LM repeated out shape torch.Size([1, 1, 1024])
fused output= torch.Size([1, 1, 2048])
single out= False
x= torch.Size([1, 1, 2048])
{'camera': tensor([[[[-4.8138, -4.9176, -5.1456, -4.7176, -4.8106, -5.0663, -5.2474,
           -5.2863, -5.2326, -5.1281, -4.8281, -4.8934, -4.8535, -4.7908,
           -4.8386, -5.0008, -4.5760, -4.3105, -5.2323, -4.6200, -4.9103,
           -5.0414, -5.0926, -5.0831, -4.6425, -4.0986, -4.7054, -4.6924,
           -4.9182, -4.7041, -4.4830, -5.1584, -5.0706, -4.4771, -5.0330,
           -4.2649, -4.6336, -4.7563, -5.1932, -4.7054, -5.0218, -4.6650,
           -5.1092, -4.2963, -5.2811, -5.1195, -5.0490, -4.9329, -4.6116,
           -4.8269, -4.9860, -4.7757, -5.0068, -4.7756, -4.8594, -4.5379,
           -4.4097, -5.3564, -5.1726, -5.1785, -4.7006, -4.9684, -4.6883,
           -5.1529, -5.1156, -4.8982, -5.3304, -4.7692, -4.9016, -4.819

In [5]:
print(VPT_state_out[0])#agent.policy.initial_state(1)[0])

(tensor([[[False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, F

In [6]:
th.arange(1)

tensor([0])

In [7]:
from inverse_dynamics_model import IDMAgent

agent_parameters = pickle.load(open('4x_idm.model', "rb"))
net_kwargs = agent_parameters["model"]["args"]["net"]["args"]
pi_head_kwargs = agent_parameters["model"]["args"]["pi_head_opts"]
pi_head_kwargs["temperature"] = float(pi_head_kwargs["temperature"])
agent = IDMAgent(device=DEVICE, idm_net_kwargs=net_kwargs, pi_head_kwargs=pi_head_kwargs)


: 

: 

In [ ]:
first = th.from_numpy(np.array((False,))).to(DEVICE)
state = policy.initial_state(1)
print(len(state))

ob_img = {"pov": np.random.normal(0,0.02,(128,128,3)) }
ob_img = agent._env_obs_to_agent( ob_img )
print("ob_img shape:",ob_img["img"].shape)

ob_words = agent._words_to_agent("hi, this is a test sentence ... test1 ... hello there ... test 3.")
print(ob_words['token_ids'].shape)



action_prob, action, pd_word, state_out = policy.get_output_for_observation(ob_words=ob_words, ob_img=ob_img, first=first, VPT_state_in=state) # test for frames=1 and n_frames=2




In [ ]:

ob_frames = [np.ones((128,128,3))*0.01 for i in range(10)]
ob_frames = agent._video_obs_to_agent( ob_frames )
ob_frames['img'][0,:,:,:,:] = th.ones([10,128,128,3])*-0.05
ob_words = agent._words_to_agent("I'm going to count to 10: 1, 2, 3, 4, 5, 6, 7, 8, ")

policy.net.Xattn_VPT_LM.alpha_xattn=th.nn.Parameter(th.tensor(0.))
policy.net.Xattn_VPT_LM.alpha_xattn=th.nn.Parameter(th.tensor(0.))

action_prob, action, pd_word = policy.get_output_for_observations(ob_words=ob_words, ob_frames=ob_frames) # test for frames=1 and n_frames=2
print(pd_word.mean(), pd_word.std())


token_index = th.argmax(pd_word[0,-1,:]).item()

print(agent._agent_words_to_string(token_index))

In [ ]:


EPOCHS = 20
# Needs to be <= number of videos
BATCH_SIZE = 8
# Ideally more than batch size to create
# variation in datasets (otherwise, you will
# get a bunch of consecutive samples)
# Decrease this (and batch_size) if you run out of memory
N_WORKERS = 12
LOSS_REPORT_RATE = 100
LEARNING_RATE = 0.000181
WEIGHT_DECAY = 0.039428
MAX_GRAD_NORM = 5.0
# Basic behavioural cloning
# Note: this uses gradient accumulation in batches of ones
#       to perform training.
#       This will fit inside even smaller GPUs (tested on 8GB one),
#       but is slow.
# NOTE: This is _not_ the original code used for VPT!
#       This is merely to illustrate how to fine-tune the models and includes
#       the processing steps used.                                               @FIX THIS to run on 4x3090 = 96GB:

# This will likely be much worse than what original VPT did:
# we are not training on full sequences, but only one step at a time to save VRAM.
def behavioural_cloning_train(data_dir, in_model, in_weights, out_weights):
    agent_policy_kwargs, agent_pi_head_kwargs = load_model_parameters(in_model)

    # To create model with the right environment.
    # All basalt environments have the same settings, so any of them works here
    env = gym.make("MineRLBasaltFindCave-v0")
    agent = MineRLAgent(env, device=DEVICE, policy_kwargs=agent_policy_kwargs, pi_head_kwargs=agent_pi_head_kwargs)
    agent.load_weights(in_weights)
    env.close()

    policy = agent.policy
    trainable_parameters = policy.parameters()

    # Parameters taken from the OpenAI VPT paper
    optimizer = th.optim.Adam(
        trainable_parameters,
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )

    data_loader = DataLoader(
        dataset_dir=data_dir,
        n_workers=N_WORKERS,
        batch_size=BATCH_SIZE,
        n_epochs=EPOCHS
    )

    start_time = time.time()

    # Keep track of the hidden state per episode/trajectory.
    # DataLoader provides unique id for each episode, which will
    # be different even for the same trajectory when it is loaded
    # up again
    episode_hidden_states = {}
    dummy_first = th.from_numpy(np.array((False,))).to(DEVICE)

    loss_sum = 0
    for batch_i, (batch_images, batch_actions, batch_episode_id) in enumerate(data_loader):
        batch_loss = 0
        for image, action, episode_id in zip(batch_images, batch_actions, batch_episode_id):
            
            agent_action = agent._env_action_to_agent(action, to_torch=True, check_if_null=True)
            
            if agent_action is None:
                # Action was null
                continue

            agent_obs = agent._video_obs_to_agent({"pov": image})

            if episode_id not in episode_hidden_states:
                # TODO need to clean up this hidden state after worker is done with the work item.
                #      Leaks memory, but not tooooo much at these scales (will be a problem later).
                episode_hidden_states[episode_id] = policy.initial_state(1)
            agent_state = episode_hidden_states[episode_id]
            
            pi_distribution, v_prediction, new_agent_state, word_pd, word_v = policy.get_output_for_observations(
                ob_frames = agent_obs,
                ob_words = ob_words,
                VPT_state_in = agent_state,
                dummy_first = None
            )

            if (LM_full):
                loss = 


            # ACTION LOSS
            log_prob  = policy.get_logprob_of_action(pi_distribution, agent_action)


            # LANGUAGE LOSS - from modeling_opt.py
            logits = word_pd.contiguous()
            labels = th.full([word_pd.shape[0],word_pd.shape[1],1], -100, dtype=th.LongTensor) # dont compute loss for masked tokens (hence -100 token ids for masked tokens, as per OPT)
            for b in range(len(ob_words['input_ids'])):
                n_tokens = len(ob_words['input_ids'][b])
                labels[:n_tokens] = ob_words['input_ids']
            loss = None
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()      #@ make sure to not train 
            # Flatten the tokens
            loss_fct = th.nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, policy.net.LM.model.decoder.vocab_size), shift_labels.view(-1))








            # Make sure we do not try to backprop through sequence
            # (fails with current accumulation)
            new_agent_state = tree_map(lambda x: x.detach(), new_agent_state)
            episode_hidden_states[episode_id] = new_agent_state

            # Finally, update the agent to increase the probability of the
            # taken action.
            # Remember to take mean over batch losses
            loss = -log_prob / BATCH_SIZE
            batch_loss += loss.item()
            loss.backward()

        th.nn.utils.clip_grad_norm_(trainable_parameters, MAX_GRAD_NORM)
        optimizer.step()
        optimizer.zero_grad()

        loss_sum += batch_loss
        if batch_i % LOSS_REPORT_RATE == 0:
            time_since_start = time.time() - start_time
            print(f"Time: {time_since_start:.2f}, Batches: {batch_i}, Avrg loss: {loss_sum / LOSS_REPORT_RATE:.4f}")
            loss_sum = 0

    state_dict = policy.state_dict()
    th.save(state_dict, out_weights)




"""
attention_heads': 16,
  'attention_mask_style': 'clipped_causal',
  'attention_memory_size': 256,
  'diff_mlp_embedding': False,
  'hidsize': 2048,
  'img_shape': [128, 128, 3],
  'impala_chans': [16, 32, 32],
  'impala_kwargs': {'post_pool_groups': 1},
  'impala_width': 8,
  'init_norm_kwargs': {'batch_norm': False, 'group_norm_groups': 1},
  'n_recurrence_layers': 4,
  'only_img_input': True,
  'pointwise_ratio': 4,
  'pointwise_use_activation': False,
  'recurrence_is_residual': True,
  'recurrence_type': 'transformer',
  'timesteps': 128,
  'use_pointwise_layer': True,
  'use_pre_lstm_ln': False},

  """

In [ ]:
attention = th.nn.MultiheadAttention(
            embed_dim=512,  # in pytorch, its split across heads
            num_heads=16,
            dropout=0.0,
            bias=True,
            add_bias_kv=False,
            add_zero_attn=False,
            kdim=128*16,vdim=128*16,
            batch_first=True,
            device=DEVICE,
            dtype=th.float32)

tokensL = th.normal(0,0.02,[2,4,512])
tokensV = th.normal(0,0.02,[2,5,2048])

attn_mask = th.zeros([4,5])
attn_mask[-1,:] = 1
print(attn_mask)

out = attention(query=tokensL, key=tokensV, value=tokensV, attn_mask=attn_mask)
print(out)
th.isnan(out[0]).any()